### Key Steps for Exploratory Data Analysis (EDA)
* Reading dataset
* Analyzing the data
* Checking for the duplicates
* Missing Values Calculation
* Analyzing the dataset:
* Univariate Analysis
* Bivariate Analysis
* Multivariate Analysis

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings as wr
wr.filterwarnings('ignore')

df = pd.read_csv("Datasets\Tech_Stock_Data_2019-2025.csv")
print(df.head())

         Date      NVDA        AMD       INTC      GOOGL       MSFT  \
0  2019-01-01       NaN        NaN        NaN        NaN        NaN   
1  2019-01-02  3.377354  18.830000  40.518055  52.372784  94.612602   
2  2019-01-03  3.173305  17.049999  38.289036  50.922279  91.131996   
3  2019-01-04  3.376611  19.000000  40.638535  53.534267  95.370476   
4  2019-01-05       NaN        NaN        NaN        NaN        NaN   

          CRM       ORCL       OKTA         ZS  ...  VIX_MA20  VIX_vs_MA  \
0         NaN        NaN        NaN        NaN  ...       NaN        NaN   
1  134.139008  40.754517  63.259998  39.810001  ...       NaN        NaN   
2  129.042572  40.357971  59.560001  39.320000  ...       NaN        NaN   
3  136.523895  42.097385  63.150002  41.770000  ...       NaN        NaN   
4         NaN        NaN        NaN        NaN  ...       NaN        NaN   

   Vol_of_Vol_Ratio  Semi_vs_Tech_Ratio  Small_vs_Large_Caps  NASDAQ_MA5  \
0               NaN                 NaN 

In [7]:
df.shape
print("\n====Detailed Overview=====")
df.info()



====Detailed Overview=====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2516 entries, 0 to 2515
Columns: 103 entries, Date to Credit_Spread_Proxy
dtypes: float64(84), int64(18), object(1)
memory usage: 2.0+ MB


In [8]:
df.columns.tolist()

['Date',
 'NVDA',
 'AMD',
 'INTC',
 'GOOGL',
 'MSFT',
 'CRM',
 'ORCL',
 'OKTA',
 'ZS',
 'CRWD',
 'DDOG',
 'NET',
 'AAPL',
 'META',
 'AMZN',
 'IBM',
 'ADBE',
 'NOW',
 'SHOP',
 'TWLO',
 'MDB',
 'PYPL',
 'ANET',
 'PANW',
 'SP500',
 'NASDAQ',
 'Dow_Jones',
 'Russell2000',
 'VIX',
 'VVIX',
 'NASDAQ_VIX',
 'Treasury_10Y',
 'Treasury_3M',
 'Treasury_5Y',
 'Treasury_30Y',
 'Yield_Curve_Slope',
 'Yield_Curve_Inverted',
 'Semiconductor_ETF',
 'AI_Robotics_ETF',
 'Cybersecurity_ETF',
 'ARK_Autonomous_Tech',
 'Taiwan_ETF',
 'Cloud_Computing_ETF',
 'Software_ETF',
 'South_Korea_ETF',
 'NASDAQ_100_ETF',
 'China_Large_Cap_ETF',
 'Global_Robotics_ETF',
 'Real_Estate_ETF',
 'Tech_Sector_ETF',
 'Global_Cloud_ETF',
 'Japan_ETF',
 'ARK_Next_Gen_Internet',
 'ARK_Innovation',
 'Dollar_Index',
 'Dollar_Strength',
 'Gold',
 'Silver',
 'Copper',
 'Oil_WTI',
 'Natural_Gas',
 'Gold_Oil_Ratio',
 'Bitcoin',
 'Ethereum',
 'Crypto_Tech_Corr_20d',
 'Pandemic_Period',
 'AI_Boom_Period',
 'Fed_Hike_Period',
 'Tech_Bear

In [9]:
df.nunique()

Date                   2516
NVDA                   1721
AMD                    1630
INTC                   1615
GOOGL                  1711
                       ... 
NASDAQ_MA5              294
NASDAQ_MA20               0
NASDAQ_Momentum           0
Dollar_MA20               0
Credit_Spread_Proxy    1733
Length: 103, dtype: int64

In [3]:
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
df.set_index("Date", inplace=True)

### Univariate Analysis

In [11]:
df.describe()

,NVDA,AMD,INTC,GOOGL,MSFT,CRM,ORCL,OKTA,ZS,CRWD,...,VIX_MA20,VIX_vs_MA,Vol_of_Vol_Ratio,Semi_vs_Tech_Ratio,Small_vs_Large_Caps,NASDAQ_MA5,NASDAQ_MA20,NASDAQ_Momentum,Dollar_MA20,Credit_Spread_Proxy
count,1733.000000,1733.000000,1733.000000,1733.000000,1733.000000,1733.000000,1733.000000,1733.000000,1733.000000,1622.000000,...,0.0,0.0,1725.000000,1733.000000,1733.000000,294.000000,0.0,0.0,0.0,1733.000000
mean,47.900993,98.261448,38.832704,119.984913,285.457588,216.970655,97.475365,130.124743,167.481021,217.579436,...,NaN,NaN,5.354141,0.891305,0.454944,13454.367840,NaN,NaN,NaN,0.657674
std,53.580672,47.089996,11.045563,47.794091,113.970381,52.667203,55.488603,64.291771,75.670943,121.416443,...,NaN,NaN,1.123853,0.135921,0.062077,3991.609257,NaN,NaN,NaN,0.148985
min,3.173305,17.049999,18.129999,50.922279,91.131996,123.006073,36.643833,45.020000,39.320000,33.009998,...,NaN,NaN,2.054459,0.677251,0.346835,6927.020020,NaN,NaN,NaN,0.359187
25%,12.336245,66.300003,29.256401,76.831589,202.020081,165.212097,53.353050,83.199997,112.269997,128.520000,...,NaN,NaN,4.512173,0.772029,0.399018,10795.564990,NaN,NaN,NaN,0.541143
50%,20.657600,94.559998,41.241623,117.969360,272.426819,215.166611,79.704979,101.160004,169.630005,190.940002,...,NaN,NaN,5.527798,0.871637,0.457192,13306.102930,NaN,NaN,NaN,0.613796
75%,79.072350,125.269997,47.764519,147.031921,387.890564,255.063278,115.224899,182.800003,211.429993,292.464996,...,NaN,NaN,6.207071,1.004691,0.508247,15823.710547,NaN,NaN,NaN,0.799461
max,207.039993,264.329987,62.083336,292.809998,541.057373,365.656708,327.761200,291.779999,368.779999,557.530029,...,NaN,NaN,8.019528,1.218272,0.596649,23745.904687,NaN,NaN,NaN,0.937015
